In [1]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
# using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
# using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
using Statistics
# using JuliaDB
# using Wavelets,WaveletsExt
# using LinearAlgebra
# import Pkg; Pkg.add("ShiftedArrays")
# using ShiftedArrays
# include("utils.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9703404309746482301\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-5225893566980368987\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9703404309746482301\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [30]:
json_data_folder = "../user_data/data/binance"
json_filename = "ETH_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [3]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

228592

In [31]:
start_date = DateTime(2021, 3, 1)
end_date = DateTime(2021, 6, 9)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

143477

In [11]:
all([1,2,3] .> [0,1,1])

true

In [37]:
function backtest(sl_slider, tp_slider, negated_slider, cooloff_slider, ema_1, ema_1_ch, ema_2, ema_2_ch)
#     println("$sl_slider, $lg_thresh_slider, $tp_slider")
    capital = 10000.
    risk_percent = 0.02
    qfee = 0.001
    

    dfo = DataFrame(ut = Int64[], 
            tt = DateTime[],
            order = Symbol[],
            oix = Int64[],
            reason = Symbol[],
            price = Float64[],
#             target = Float64[],
            stop = Float64[],
            qty = Float64[],
            pnl = Float64[],
            capital = Float64[]
        )

    push!(dfo, (ut = ut[1], tt=tt[1], oix =0, order = :long, reason = :toolong, price = 0, stop = 0,
                qty = 0, pnl = 0, capital = capital))


    in_long_pos = false 
    qlsellsig = 1.01
    qstop = 0.01
    in_i = 0
    for i in 300:length(cc)

        ord = last(dfo)
        if !in_long_pos
            if true &&
                (ord.pnl > 0 || i - ord.oix > cooloff_slider) &&
                cc[i-1] < ema_2_ch[i-1] && cc[i] > ema_2_ch[i] 
#                 ema_1[i] < ema_2_ch[i] 
                
#                 stop_price = cc[i] * (1 - sl_slider/100)
                stop_price = minimum(cc[i-30:i])* (1 - sl_slider/100)


#                 stop_price = cc[i] * (1 - sl/100) # * (1-qfee)

                risk_size = capital * risk_percent

                buy_price = cc[i]*(1+qfee)

                risk_price_range = (buy_price - stop_price)
                buyq = risk_size / risk_price_range

#                 target = cc[i] * (1+target_pe/100)  # 2*risk_price_range

                push!(dfo, (ut = ut[i], tt=tt[i], oix=i, order = :long, reason = :enter, price = buy_price, qty = buyq,
                        stop = stop_price, pnl = 0, capital = capital))

                in_long_pos = true
            end
        else
#             sell = false
            if cc[i] > ord.price * (1 + tp_slider/100)
                ord.stop = cc[i]
#             elseif i - ord.oix > negated_slider && all(cc[i-15:i] .< ema_2[i-15:i]) && ord.stop < ord.price
#                 ord.stop = ema_2[i]
#             elseif cc[i]
#             elseif cc[i-1] < ema_2[i-1] && cc[i] > ema_2[i] && ord.stop < ord.price
#                 ord.stop = ord.price
#             elseif minimum(cc[i-30:i]) > ord.stop
#                 ord
#             elseif minimum(cc[i-26*2]) < minimum(cc[i-26*1]) && 
#                 cc[i] > 1.005 * minimum(cc[i-26*2]) &&
#                 ord.stop < minimum(cc[i-26*2])
                
#                 ord.stop = minimum(cc[i-26*2])
                
            end
            
            if cc[i] <= ord.stop
                sell_price = cc[i]*(1-qfee)
                profit_loss = ord.qty * (sell_price - ord.price)
                capital = capital + profit_loss

                push!(dfo, (ut = ut[i], tt=tt[i], oix=i, order = :long, reason = :exit, price = sell_price, stop = 0, 
                         qty = ord.qty, pnl = profit_loss, capital = capital))

                in_long_pos = false
            end
           

        end
    #     println(capital)

    end
    CSV.write("trades.csv", dfo)
    dfo
end

backtest (generic function with 1 method)

In [39]:
ar_len = length(cc)
plot_box = Observable{Any}(dom"div"())
res_box = Observable{Any}(dom"div"())
window_size = 60 * 8 * 3 * 7
num_windows = Int64(trunc(ar_len/window_size*1))
window_no_slider = slider(1:num_windows; label="start", default=7)
window_start = Interact.@map max(1, Int(&window_no_slider * window_size / 1))
window_end = Interact.@map min(ar_len, &window_start+window_size-1)
rng = Interact.@map &window_start: &window_end
put = Interact.@map ut[&rng]
wtt = Interact.@map tt[&rng]

ema_1_slider = slider(0:10:400; label="ema1", default=50)
ema_1 = Interact.@map ema(cc, n=&ema_1_slider)
ema_1_x_slider = slider(0:5:240; label="ema1X", default=0)
ema_1_y_slider = slider(-5:0.1:5; label="ema1Y", default=0)
ema_1_trans = Interact.@map circshift((&ema_1 .* (1 + &ema_1_y_slider/100)), &ema_1_x_slider)
ema_1_ch_slider = slider(-5:0.1:1; label="ema1ch", default=-1)
ema_1_ch = Interact.@map &ema_1_trans .* (1 + &ema_1_ch_slider / 100)

ema_2_slider = slider(0:10:400; label="ema2", default=50)
ema_2 = Interact.@map ema(cc, n=&ema_2_slider)
ema_2_x_slider = slider(0:5:240; label="ema2X", default=0)
ema_2_y_slider = slider(-5:0.1:5; label="ema2Y", default=0)
ema_2_trans = Interact.@map circshift((&ema_2 .* (1 + &ema_2_y_slider/100)), &ema_2_x_slider)
ema_2_ch_slider = slider(-5:0.1:1; label="ema2ch", default=-1)
ema_2_ch = Interact.@map &ema_2_trans .* (1 + &ema_2_ch_slider / 100)

negated_slider = slider(0:600; label="neg", default=-1)
cooloff_slider = slider(0:5:600; label="cool", default=-1)
# log_ema_slider = slider(0:100; label="emalog", default=7)
# log_val = Interact.@map log.(cc ./ &ema_1) .* 1000
# log_ema = Interact.@map ema(&log_val, n=&log_ema_slider)
# lg_thresh_slider = slider(-2:0.1:5; label="tsh", default=0.1)

sl_slider = slider(0:0.2:5; label="sl", default=0.1)
tp_slider = slider(0:0.1:5; label="tp", default=1)
# look_back_slider = slider(0:5:240; label="lb", default=10)


dfo = Interact.@map backtest(&sl_slider, &tp_slider, &negated_slider, &cooloff_slider, &ema_1_trans, &ema_1_ch, &ema_2_trans, &ema_2_ch)

wdfo = Interact.@map filter(row -> (row.ut >= ut[&window_start] && row.ut <= ut[&window_end]), &dfo, view=false)

# res_capital = Interact.@map trunc(last(&wdfo).capital)

buys = Interact.@map filter(row -> row.reason == :enter, &wdfo)
losses = Interact.@map filter(row -> row.reason == :exit && row.pnl < 0, &wdfo)
profits = Interact.@map filter(row -> row.reason == :exit && row.pnl > 0, &wdfo)
no_losses = Interact.@map nrow(&losses)
no_profits = Interact.@map nrow(&profits)
tot_losses = Interact.@map Int(trunc(sum((&losses).pnl)))
tot_profits = Interact.@map Int(trunc(sum((&profits).pnl)))


wbuys = Interact.@map filter(row -> row.reason == :enter, &wdfo)
wlosses = Interact.@map filter(row -> row.reason == :exit && row.pnl < 0, &wdfo)
wprofits = Interact.@map filter(row -> row.reason == :exit && row.pnl > 0, &wdfo)

# , buys, profits, losses
function draw_it_all(rng, ema_1, ema_1_ch, ema_2, ema_2_ch, buys, profits, losses)
    put = ut[rng]   
    
    f1 = plot(put, cc[rng], size=(1200,500))
    plot!(put,ema_1[rng], label="ema1",lw=0.8)
    plot!(put,ema_2[rng], label="ema2",lw=0.8)
    
#     plot!(put,ema_1[rng], fillrange=ema_1_ch[rng], fillalpha=0.15, label="ch1",lw=0)
    plot!(put,ema_2[rng], fillrange=ema_2_ch[rng], fillalpha=0.15, label="ch2",lw=0)
    
    
    nrow(buys) > 0 && vline!(buys.ut,lw=0.5, color=:blue)
    nrow(profits) > 0 && vline!(profits.ut,lw=0.5, color=:green)
    nrow(losses) > 0 && vline!(losses.ut,lw=0.5, color=:red)
    
#     f2 = plot(put, log_val[rng])
#     plot!(put, log_ema[rng])
#     hline!(-10,lw=1)
    
#     plt = plot(f1, f2, layout=@layout[a{0.6h}; b{0.25h};], size=(1400,500))
    dom"div"(vbox(f1))
end
# , wbuys, wprofits, wlosses
map!(draw_it_all, plot_box, rng, ema_1_trans, ema_1_ch, ema_2_trans, ema_2_ch, wbuys, wprofits, wlosses)

ema_1_slider[]=90
ema_1_ch_slider[]=-0.6
ema_1_x_slider[] = 0
ema_1_y_slider[] = -2.30

ema_2_slider[]=100
ema_2_ch_slider[]=-0.6
ema_2_x_slider[] = 0
ema_2_y_slider[] = 0

negated_slider[] = 222
cooloff_slider[] = 310
sl_slider[] = 3.4
tp_slider[]=3.3
# lg_thresh_slider[] = 1
# look_back_slider[] = 5
# dff = dateformat"Y-m-d H:M:S"
dff = dateformat"mm/dd"
ui = dom"div"(hbox(
            vbox(ema_1_slider, ema_1_ch_slider, ema_1_x_slider, ema_1_y_slider),
            vbox(ema_2_slider, ema_2_ch_slider, ema_2_x_slider, ema_2_y_slider),
            vbox(sl_slider,tp_slider,negated_slider,cooloff_slider),
#         ,lg_thresh_slider,look_back_slider
#             vbox(log_ema_slider,sl_slider),
            Interact.@map vbox( (&no_losses,&no_profits),
                                    (&tot_losses, &tot_profits),
                                    (&tot_losses + &tot_profits))
        ),
        plot_box,window_no_slider,
        Interact.@map vbox(("$(Dates.format(first(tt),dff))","$(Dates.format(last(tt),dff))"), 
            ("$(Dates.format(first(&wtt),dff))","$(Dates.format(last(&wtt),dff))"))
        )
# using Interact, Blink
# w = Window()
# body!(w, ui);
ui
# 1

(div
  (div { style=Dict("display" => "flex", "flex-direction" => "row") }
    (div { style=Dict("display" => "flex", "flex-direction" => "column") }
      Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ema1"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 41, :default => 50, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
10, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000111eb1aa0, Task (runnable) @0x0000000111eb1aa0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0\",\"10\",\"20\",\"30\",\"40\",\"50\",\"60\",\"70\",\"80\",\"90\",\"100\",\"110\",\"120\",\"130\",\"140\",\"150\",\"160\",\

In [8]:
using Dates
dff = dateformat"Y-m-d H:M:S"
Dates.format(now(),dff)

"2021-6-9 5:32:3"

In [40]:
last(dfo[])

,ut,tt,order,oix,reason,price,stop,qty
,Int64,DateTime,Symbol,Int64,Symbol,Float64,Float64,Float64
387,1623160380,2021-06-08T13:53:00,long,142870,exit,1.49241,0.0,5244.08
